# -----------------------------------------------------------------------------
# EXPLORING SEISMOGRAMS

This is a script that allows you to plot a range of data.  
At the top there are a number of parameteres to change:
- What compontents of data to plot
- What frequencies to plot
- What phase to center on
- Which travel times to plot
- Organizing seismograms as a function of distance or azimuth

Next, it will allow you to look at the ray paths of the phase you are focusing on, and create a map of the event-station geometry. 

Code blocks can be run with shift-enter.
NOTE: every time you change something, you need to re-run  that code-block, before rerunnig the last code block (which does the actual plotting)

#### Start with exploring the plotting setup and then go to the exercises listed at the bottom. 

# -----------------------------------------------------------------------------
# IMPORTS
These blocks only need to be run once (if all goes well)
## Importing libraries

In [ ]:
%matplotlib notebook

import obspy
import obspy.taup
from obspy import read
from obspy import UTCDateTime
from obspy.taup import TauPyModel
import obspy.signal
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import numpy as np
import sys

import cartopy
import cartopy.crs as ccrs

import glob
import time

plt.rcParams.update({'font.size': 12})

## Importing data
The data set is for a March 21st 2010 event in Papua New Guinea, recorded across stations in the US. 

In [ ]:
seislist = glob.glob('Data/*')
allseis = []
for i in range(len(seislist)):
    allseis.append(obspy.read(seislist[i],'PICKLE'))
print(len(allseis), ' seismograms were loaded')

model = TauPyModel(model='ak135') # Load 1D model for travel time and ray predictions


# -----------------------------------------------------------------------------
# PLOTTING WAVEFORMS


## Phase to center plot on
Choose a phase name to plot. One backup phase name can be be provided, if first phase doesn't exist at a specific distance, secondary phase will be tried, e.g. this way direct S which turn into S-diffracted waves with distance can be plot. <br>
Phases follow taup naming conventions, see:  <br>
https://docs.obspy.org/packages/obspy.taup.html <br>

In [ ]:
center_phase = ['S', 'Sdiff']

## Components and subplots to plot.


Choose 'BHT' for transverse, 'BHR' for radial, or 'BHZ' for vertical. This sets the number of subplots, the components will be plotted side by side.  

In [ ]:
components = ['BHR', 'BHT']

## Predicted travel times to plot
Phases follow taup naming conventions, see:  <br>
https://docs.obspy.org/packages/obspy.taup.html <br>
A number of phases are precomputed and stored with the data, others will slow down the script to compute on the fly

In [ ]:
tt_phases = ['S', 'Sdiff', 'ScS']

## Set frequency limits
or 1/period  <br>
Frequencies up to 0.5 Hz are included. <br>
Note, the fmax is 1/17 Hz for the shakemovie synthetics. <br>

In [ ]:
fmin = 1. / 30. #1/period
fmax = 1. / 10. #1/period

## Y-axis: Plot as function of distance or azimuth
Chooose to plot with distance or azimuth <br>
Choose 'dist' or 'az' <br>
'round' sets if seismograms are plotted at values rounded to whole numbers (this generally makes the plot easier to look at). 

In [ ]:
yaxis = 'dist'
round = True

## Limits for azimuth, distance and time
- Azimuth limits (data is included between 40 and 70 deg) <br>
- Distance limits (data is included between 85 and 110 deg) <br>
- Time limits (Window around center phase)

In [ ]:
# in dg
azmin = 40.
azmax = 70.

# epicentral distance in dg
distmin = 85
distmax = 110

# in sec
timemin = -80
timemax = 120

## Normalization factor
factor to normalize all data by to improve plot <br>

In [ ]:
fact = 1.

## Plotting data
All the plotting happens after this, and in principle doesn't
need to be changed... although you are welcome to play
around/ improve/ break it...

In [ ]:

# Colormap for travel times
cNorm = colors.Normalize(vmin=0, vmax=len(tt_phases))
tt_cmap = cm.ScalarMappable(norm=cNorm, cmap=plt.get_cmap('jet'))




slon = []
slat = []
elon = []
dists =[]
axes = []



fig = plt.figure(figsize = (10,6))



for plotnum in range(len(components)):
     axes.append(plt.subplot(1, len(components), 1+plotnum))   
# Loop through data types
for plotnum, comp in enumerate(components):
        print('plotting ', comp, '...')

        # Loop through seismic receivers
        for s in allseis:
            seis = s.copy()
            stats = seis.select(channel='BHZ')[0].stats
            if stats['az'] > azmin and stats['az'] < azmax:
                if stats['dist'] > distmin and stats['dist'] < distmax:
                    try:
                        center_time = stats.traveltimes[center_phase[0]]
                    except:
                        arrs = model.get_travel_times(stats['evdp'],stats['dist'], phase_list = [center_phase[0]])
                        center_time = arrs[0].time

                    if len(center_phase) > 1:
                
                        if center_time is None:
                            try:
                                center_time = stats.traveltimes[center_phase[1]]
                            except:
                                arrs = model.get_travel_times(stats['evdp'],stats['dist'], phase_list = [center_phase[1]])
                                center_time = arrs[0].time
                    if center_time is None:
                        continue
        
                    seis= seis.select(channel=comp)[0]
                        
                    seis.filter('highpass',freq=fmin,corners=2,zerophase=True)
                    seis.filter('lowpass',freq=fmax,corners=2,zerophase=True)

                    xplot = seis.times(
                        reftime=UTCDateTime(
                            stats['eventtime'])) - center_time
                    win1 = np.argmin(np.abs(xplot - timemin))
                    win2 = np.argmin(np.abs(xplot - timemax))
                                                    
                    if round:
                        yloc = np.round(stats[yaxis])
                    else:
                        yloc = stats[yaxis]
                    relnorm = np.max(np.abs(seis.data[win1:win2]))
                    yplot = fact * seis.data / (relnorm) + yloc
                    axes[plotnum].plot(xplot[win1:win2], yplot[win1:win2], 'k', linewidth=1)

                
                
                    # Plot travel times
                    for ph, phase in enumerate(tt_phases):
                            if phase in stats.traveltimes:
                                if stats.traveltimes[phase] is not None:
                                    xtime = stats.traveltimes[phase] - center_time

                                    for i in range(len(components)):
                                        axes[i].plot([xtime, xtime], [yloc - 0.5,yloc + 0.5],
                                             color=tt_cmap.to_rgba(ph), linewidth=2)
                            else:
                                arrs = model.get_travel_times(stats['evdp'],stats['dist'], phase_list = (phase,))
                                if len(arrs)>0:
                                    for arr in arrs:
                                        xtime = arr.time - center_time
                                        for i in range(len(components)):
                                            axes[i].plot([xtime, xtime], [yloc - 0.5,yloc + 0.5],
                                                 color=tt_cmap.to_rgba(ph), linewidth=2)
                                       
                    #Save locations to plot
                    if elon == []:
                        elon = stats['evlo']
                        elat = stats['evla']
                        edep = stats['evdp']
                    slon.append(stats['stlo'])
                    slat.append(stats['stla'])
                    dists.append(stats['dist'])
                        

# plot some fake points to make an okay legend
for ph, phase in enumerate(tt_phases):
    plt.plot(0, 0, '.', color=tt_cmap.to_rgba(ph), label=phase)
plt.legend(bbox_to_anchor=(0.92, 0.3))


# prettify plots
for i in range(len(components)):
    axes[i].set_xlabel('time (s) w.r.t.' + center_phase[0])
    if yaxis == 'dist':
        if i ==0:
            axes[i].set_ylabel('distance (dg)')
        axes[i].set_ylim([distmin, distmax])
        axes[i].set_title(str(azmin) + ' < azimuth < ' + str(azmax))
    if yaxis == 'az':
        if i ==0:
            axes[i].set_ylabel('azimuth (dg)')
        axes[i].set_ylim([azmin, azmax])
        axes[i].set_title(str(distmin) + ' < distance < ' + str(distmax))
    axes[i].set_xlim([timemin, timemax])
    axes[i].grid()



# -----------------------------------------------------------------------------
# TRAVEL TIME AND RAYPATH TOOLS

## Predicting common phase arrivals

For a set distance, this predicts arrival times for common phases. 

In [ ]:
ref_dist = 90 #degrees
depth_earthquake=edep# depth of earthquake in km
arrs = model.get_travel_times(depth_earthquake, distance_in_degree=ref_dist, phase_list=('ttall', ))
print(arrs)

## Plot ray paths

This bit of code will plot the ray paths for the phases for which travel times are included in your data plot above. 

In [ ]:
fig= plt.figure(figsize = (10,6))
ax = fig.add_subplot(1, 1, 1, polar=True)
# Phases to plot, e.g.  tt_phaces, center_phase, or a user-defined list ["PKJKP", "SKKS"]
plot_phases = tt_phases

widths = [1,1,1,1,1,1]
# depth of earthquake in km
depth_earthquake=edep
# Distances to compute, currently set to plot a ray at the minimum and maximum distance included in the data set.
dist_raypaths = [np.min(dists),  np.max(dists)]


# computing ray paths
for d, dist in enumerate(dist_raypaths):
    if d == 0:
        arrivals= model.get_ray_paths(depth_earthquake, dist, phase_list= plot_phases)
    else:
        arrivals= arrivals+model.get_ray_paths(depth_earthquake, dist, phase_list= plot_phases)


# plotting

arrivals.plot_rays(ax = ax, legend = True, plot_all=False)



# -------------------------------------------------------------------------------
# PLOT TOMOGRAPHIC MODEL AND PIERCE POINTS
## Load depth slice
Model provided to plot on the map is S362ANI+M (Moulik & Ekstrom, 2014).<br>
Choose which depth to plot. This can be 25, 50, 75, 100, 125, 150, 200, 250, 300, 350, 400, 500, 600, 700, 800, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 2800, or 2890 km. <br>
Choose what to plot 'vsv', 'vsh', or 'voigt'. <br>
Choose 'absolute' or 'relative'. 


In [ ]:
# Settings
model3D = 'S362ANI+M' # No other model currently provided
depth_mod = 2800 # in km
to_plot = 'voigt'
abs_or_rel = 'relative'

# Load model
mod = np.loadtxt(model3D+'/'+model3D+'.'+to_plot+'.'+str(int(depth_mod))+'.txt', 
           dtype={'names': ('lon', 'lat', 'vel'),'formats': ('f4', 'f4', 'f4')})  

# Reshape for plotting
len1= len(np.unique(mod['lon']))
len2= len(np.unique(mod['lat']))
lon = mod['lon'].reshape((len2,len1))
lat = mod['lat'].reshape((len2,len1))
vel = mod['vel'].reshape((len2,len1))
print('model loaded')

## Compute pierce points
For given phase(s) and a depth


In [ ]:
pp_depth = depth_mod # Depth to compute pierce point at
pp_phases = center_phase # Phase(s) for piercepoints, can be set to center_phase


downgoing =[]
upgoing=[]
for d in range(len(dists)):
        # Compute ray paths
        arrs = model.get_ray_paths_geo(source_depth_in_km=edep, 
                                          source_latitude_in_deg=elat,
                                          source_longitude_in_deg=elon,
                                          receiver_latitude_in_deg=slat[d],
                                          receiver_longitude_in_deg=slon[d],
                                          phase_list=pp_phases)
        for A in arrs:
            p=A.path
            cut = np.argmax(p['depth'])
            # Only select rays that go deep enough
            if pp_depth < np.max(p['depth']):
                # Interpolate downgoing pierce point
                downgoing.append((np.interp(pp_depth, p['depth'][:cut],p['lon'][:cut] ), 
                             np.interp(pp_depth, p['depth'][:cut],p['lat'][:cut])))

                # Interpolate upgoing pierce point           
                upgoing.append((np.interp(pp_depth, p['depth'][cut:][::-1],p['lon'][cut:][::-1] ), 
                             np.interp(pp_depth, p['depth'][cut:][::-1],p['lat'][cut:][::-1])))

downgoing=np.asarray(downgoing)
upgoing=np.asarray(upgoing)
print('done computing pierce points')

## Plotting Map
Uses the selection of data from above

In [ ]:

plot_tomography = True # Choose to plot background seismic model
plot_piercepoints = True # Choose to plot pierce points computed above

# Plot map background
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(200, 25))
ax.set_global()
ax.coastlines()

# Plot tomographic model
if plot_tomography:
    if abs_or_rel == 'relative':
        ref = np.mean(vel.ravel())
        rel = (vel-ref)/ref*100.
        lim = np.ceil(np.max(np.abs(rel)))
        levels = np.linspace(-lim,lim,33)
        velmap = ax.contourf(lon,lat,rel,levels=levels,cmap=cm.seismic_r,linewidth=0, transform=ccrs.PlateCarree())
    if abs_or_rel == 'absolute':
        velmap =ax.contourf(lon,lat,vel,33, cmap=cm.viridis_r,linewidth=0, transform=ccrs.PlateCarree())    

    plt.colorbar(velmap)
    plt.title('model ' + model3D + ' sliced at ' +str(int(depth_mod)) + 'km', fontsize=18)

#Plot pierce points
if plot_piercepoints:
    ax.scatter(downgoing[:,0], downgoing[:,1],s=35,c='y',marker='o',edgecolors = 'k',alpha=1, transform=ccrs.PlateCarree())
    ax.scatter(upgoing[:,0], upgoing[:,1],s=35,c='c',marker='o',edgecolors = 'k',alpha=1, transform=ccrs.PlateCarree())

#Plot event
ax.scatter(elon,elat,s=600,marker='*',facecolors='y', edgecolors = 'k', alpha=1, transform=ccrs.PlateCarree())
#Plot stations
ax.scatter(slon,slat,s=35,c='m',marker='^',  edgecolors = 'k', alpha=1, transform=ccrs.PlateCarree())


# -------------------------------------------------------------------------------
# Exercises 

Make and optimise a data plot for each of these exercises. 
Think about changing the component, distance range and frequency content of the data in your plot, and setting your plot to plotting with distance or azimuth. 

1. Find data evidence for seismic anisotropy in the mantle. 

2. Find evidence for the presence of a ULVZ beneath Hawaii. 

3. Find energy bouncing of the inner core. 
